In [20]:
import pandas as pd
import ast
from nltk.tokenize import TweetTokenizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [21]:
raw_df = pd.read_csv('COVID-tweets.csv')
raw_df.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,quoted_status_id_str,quoted_status_permalink,reply_count,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,NaN,NaN,Fri Jan 31 23:58:59 +0000 2020,NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,{'created_at': 'Fri Jan 31 23:30:08 +0000 2020...,"<a href=""http://twitter.com/download/iphone"" r...","RT @CNN: There have been more than 9,800 cases...",False,"{'id': 292918761, 'id_str': '292918761', 'name..."
1,NaN,NaN,Fri Jan 31 23:58:59 +0000 2020,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,{'created_at': 'Fri Jan 31 22:19:00 +0000 2020...,"<a href=""http://twitter.com/download/iphone"" r...","RT @JaneLytv: 43. Zero Hedge, a pro-Trump webs...",False,"{'id': 262334207, 'id_str': '262334207', 'name..."
2,NaN,NaN,Fri Jan 31 23:58:59 +0000 2020,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,{'created_at': 'Fri Jan 31 21:08:39 +0000 2020...,"<a href=""http://twitter.com/download/iphone"" r...",RT @skarlamangla: This flu season has killed 1...,False,"{'id': 2451840349, 'id_str': '2451840349', 'na..."
3,NaN,NaN,Fri Jan 31 23:58:59 +0000 2020,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,{'created_at': 'Fri Jan 31 05:06:49 +0000 2020...,"<a href=""http://twitter.com/download/iphone"" r...",RT @jamesmassola: Medical experts raise concer...,False,"{'id': 513208128, 'id_str': '513208128', 'name..."
4,NaN,NaN,Fri Jan 31 23:58:59 +0000 2020,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,{'created_at': 'Fri Jan 31 01:27:50 +0000 2020...,"<a href=""http://twitter.com/download/iphone"" r...","RT @popplioikawa: the flu: *results in 500,000...",False,"{'id': 993138436667314176, 'id_str': '99313843..."


In [22]:
def evaluate(row):
    '''
    This function, when applied to
    the series containing 
    dictionary-like strings will 
    convert each instance to
    actual dictionaries and return
    the dictionary.
    '''
    
    row = ast.literal_eval(row) 
    return row

In [30]:
# Saving dictionaries in new column
exp_df = raw_df.copy()
exp_df['expanded'] = exp_df.retweeted_status.dropna().apply(evaluate)

# Creating features from dictionary keys in new dataframe
expanded_df = exp_df['expanded'].apply(pd.Series)

# Again, unnesting another dictionary to get to the full_text column
expanded_df = expanded_df.extended_tweet.apply(pd.Series)

# Dropping rows corresponding to original tweets (not retweeted text)
expanded_df = expanded_df.full_text.dropna()

# Joining with original dataframe
df = pd.DataFrame.join(exp_df, expanded_df)

# Swapping NaNs for original tweets in the full text column
df['full_text'].fillna(df['text'],inplace=True)

In [27]:
analyzer = SentimentIntensityAnalyzer()

def sentiment_score(tweet):
    return analyzer.polarity_scores(tweet)

In [34]:
df['sentiment'] = df['full_text'].apply(sentiment_score)

In [35]:
df['sentiment'].head()

0    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
1    {'neg': 0.096, 'neu': 0.853, 'pos': 0.05, 'com...
2    {'neg': 0.429, 'neu': 0.571, 'pos': 0.0, 'comp...
3    {'neg': 0.0, 'neu': 0.846, 'pos': 0.154, 'comp...
4    {'neg': 0.113, 'neu': 0.845, 'pos': 0.042, 'co...
Name: sentiment, dtype: object

In [36]:
df['sentiment'][1]

{'neg': 0.096, 'neu': 0.853, 'pos': 0.05, 'compound': -0.4767}

In [38]:
def polarity_score(scores):
    if scores['compound'] >= 0.05:
        return 1
    elif scores['compound'] >-.05 and scores['compound'] < 0.05:
        return .5
    else:
        return 0
    

In [39]:
df['polarity'] = df['sentiment'].apply(polarity_score)

In [41]:
df.to_csv('tweet_polarity.csv',index=False)